In [1]:
#Program: Name: Daily Build Datasets
#Purpose: Daily (Data Quality) Run for collecting details about system changes and merging-AB into Local-C
#Author:  Greg Turmel, Director, Data Governance 
#Date:    2020.08.30 - 2021.06.30
#Errata:  0.1 Improvements can be made to script using for/looping through the databases

import os, datetime
import sqlalchemy as db
from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np
import matplotlib as plt
import pyodbc 
import time

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

%matplotlib inline
%load_ext sql

In [2]:
# This segment builds the appropriatte file system structure as a variable driven exercise
# Take time to set your 'eeeeeeeeee' number as variable 'pn' below 
# and set the program directory variable called 'programDirectory' before running
# ===================================================================================================================
pn = r'eeeeeeeeee'            #This represents the windows system employee login folder - IBM team uses a 9 number
# ===================================================================================================================

programDirectory = 'Daily' # Update this variable to wherever you want the program subfolder/files to be located 
un = r'C:\Users'
cn = r'Documents\Py'
tn = r'local\T'

#Set a parent directory
parentDirectory = "{}\{}\{}".format(un,pn,cn)
print('Parent Directory is: ', parentDirectory)
mode = 0o666

#Set path location for working with local file(s)
path = os.path.join(parentDirectory, programDirectory,)
pathT = os.path.join(parentDirectory, programDirectory, tn)

try:
    if not os.path.exists(path):
        os.makedirs(path, mode)
        print('Program Directory subfolder has been created: ', programDirectory)
    else:
        print('Directory:', programDirectory, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathT):
        os.makedirs(pathT, mode)
        print('Program Directory subfolder has been created: ', pathT)
    else:
        print('Directory:', pathT, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

Parent Directory is:  C:\Users\e201873842\Documents\Py
Directory: Daily >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\local\T >>>>> Note ---- this folder already exists <<<<<


#Collect ROW counts

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ApplicationData;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountAppDataWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
   
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))


Processing Complete:  2021-05-25 07:07:00


In [4]:
#import time
time.sleep(1) # Sleep for 1 seconds
now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
print(' *** Note: tblRowCountAppDataWdata.csv file written - continue program ***')

Processing Complete:  2021-05-25 07:07:01
 *** Note: tblRowCountAppDataWdata.csv file written - continue program ***


In [5]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Assessment;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountAssmntWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:06


In [6]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountAssmntWdata.csv file written - continue program ***')

 *** Note: tblRowCountAssmntWdata.csv file written - continue program ***


In [7]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Control;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountCtrlWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:07


In [8]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountCtrlWdata.csv file written - continue program ***')

 *** Note: tblRowCountCtrlWdata.csv file written - continue program ***


In [9]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountDwnStrmFeedWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:10


In [10]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountDwnStrmFeedWdata.csv file written - continue program ***')

 *** Note: tblRowCountDwnStrmFeedWdata.csv file written - continue program ***


In [11]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds_V1.5;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountDwnStrmFeedV15Wdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:12


In [12]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountDwnStrmFeedV15Wdata.csv file written - continue program ***')

 *** Note: tblRowCountDwnStrmFeedV15Wdata.csv file written - continue program ***


In [13]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GCPS_Operations;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGCPSOpsWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:13


In [14]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGCPSOpsWdata.csv file written - continue program ***')

 *** Note: tblRowCountGCPSOpsWdata.csv file written - continue program ***


In [15]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:14


In [16]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRWdata.csv file written - continue program ***')

 *** Note: tblRowCountGSDRWdata.csv file written - continue program ***


In [17]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRSyngyWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:16


In [18]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRSyngyWdata.csv file written - continue program ***')

 *** Note: tblRowCountGSDRSyngyWdata.csv file written - continue program ***


In [19]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRSyngyTmpWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:18


In [20]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRSyngyTmpWdata.csv file written - continue program ***')

 *** Note: tblRowCountGSDRSyngyTmpWdata.csv file written - continue program ***


In [21]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRTmpWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:25


In [22]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRTmpWdata.csv file written - continue program ***')

 *** Note: tblRowCountGSDRTmpWdata.csv file written - continue program ***


In [23]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ODS_WebApps;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountODSWebAppsWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:27


In [24]:
#import time
time.sleep(1) # Sleep for 3 seconds
print(' *** Note: tblRowCountODSWebAppsWdata.csv file written - continue program ***')

 *** Note: tblRowCountODSWebAppsWdata.csv file written - continue program ***


In [25]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=PPROD;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountPPdWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:28


In [26]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountPPdWdata.csv file written - continue program ***')

 *** Note: tblRowCountPPdWdata.csv file written - continue program ***


In [27]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Predictive_Analytics;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count > 0
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountPrdtAnalyWdata.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:30


In [28]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountPrdtAnalyWdata.csv file written - continue program ***')

 *** Note: tblRowCountPrdtAnalyWdata.csv file written - continue program ***


#Find and record where Collection has ZERO records as results 

In [29]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ApplicationData;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountAppDataWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:31


In [30]:
#import time
time.sleep(3) # Sleep for 1 seconds
print(' *** Note: tblRowCountAppDataWZero.csv file written - continue program ***')

 *** Note: tblRowCountAppDataWZero.csv file written - continue program ***


In [31]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Assessment;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountAssmntWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:34


In [32]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountAssmntWZero.csv file written - continue program ***')

 *** Note: tblRowCountAssmntWZero.csv file written - continue program ***


In [33]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Control;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountCtrlWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:35


In [34]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountCtrlWZero.csv file written - continue program ***')

 *** Note: tblRowCountCtrlWZero.csv file written - continue program ***


In [35]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountDwnStrmFeedWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:36


In [36]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountDwnStrmFeedWZero.csv file written - continue program ***')

 *** Note: tblRowCountDwnStrmFeedWZero.csv file written - continue program ***


In [37]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds_V1.5;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountDwnStrmFeedV15WZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:37


In [38]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountDwnStrmFeedV15WZero.csv file written - continue program ***')

 *** Note: tblRowCountDwnStrmFeedV15WZero.csv file written - continue program ***


In [39]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GCPS_Operations;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGCPSOpsWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:39


In [40]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGCPSOpsWZero.csv file written - continue program ***')

 *** Note: tblRowCountGCPSOpsWZero.csv file written - continue program ***


In [41]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:40


In [42]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRWZero.csv file written - continue program ***')

 *** Note: tblRowCountGSDRWZero.csv file written - continue program ***


In [43]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRSyngyWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:41


In [44]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRSyngyWZero.csv file written - continue program ***')

 *** Note: tblRowCountGSDRSyngyWZero.csv file written - continue program ***


In [45]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRSyngyTmpWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:42


In [46]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRSyngyTmpWZero.csv file written - continue program ***')

 *** Note: tblRowCountGSDRSyngyTmpWZero.csv file written - continue program ***


In [47]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountGSDRTmpWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:43


In [48]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountGSDRTmpWZero.csv file written - continue program ***')

 *** Note: tblRowCountGSDRTmpWZero.csv file written - continue program ***


In [49]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ODS_WebApps;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountODSWebAppsWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:44


In [50]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountODSWebAppsWZero.csv file written - continue program ***')

 *** Note: tblRowCountODSWebAppsWZero.csv file written - continue program ***


In [51]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=PPROD;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountPPdWZero.csv'), 'w') as f:
        df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')
        

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:45


In [52]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountPPdWZero.csv file written - continue program ***')

 *** Note: tblRowCountPPdWZero.csv file written - continue program ***


In [53]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Predictive_Analytics;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
AND p.row_count < 1
ORDER BY TableName
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tblRowCountPrdtAnalyWZero.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:46


In [54]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tblRowCountPrdtAnalyWZero.csv file written - continue program ***')

 *** Note: tblRowCountPrdtAnalyWZero.csv file written - continue program ***


Collection (original) file format for winmerge analysis

In [55]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ApplicationData;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountApplicationData.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, encoding='utf-8', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:47


In [56]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountApplicationData.csv file written - continue program ***')

 *** Note: tableRowCountApplicationData.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [57]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Assessment;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountAssessment.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:48


In [58]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountAssessment.csv file written - continue program ***')

 *** Note: tableRowCountAssessment.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [59]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Control;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountControl.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:49


In [60]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountControl.csv file written - continue program ***')

 *** Note: tableRowCountControl.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [61]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountDownstreamFeeds.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:50


In [62]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountDownstreamFeeds.csv file written - continue program ***')

 *** Note: tableRowCountDownstreamFeeds.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [63]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds_V1.5;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountDownstreamFeedsV15.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:51


In [64]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountDownstreamFeedsV15.csv file written - continue program ***')

 *** Note: tableRowCountDownstreamFeedsV15.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [65]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GCPS_Operations;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountGCPS_Operations.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:52


In [66]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountGCPS_Operations.csv file written - continue program ***')

 *** Note: tableRowCountGCPS_Operations.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [67]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountGSDR_Synergy_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:54


In [68]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountGSDR_Synergy_Temp.csv file written - continue program ***')

 *** Note: tableRowCountGSDR_Synergy_Temp.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [69]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountGSDR_Synergy.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:55


In [70]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountGSDR_Synergy.csv file written - continue program ***')

 *** Note: tableRowCountGSDR_Synergy.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [71]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountGSDR_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:56


In [72]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountGSDR_Temp.csv file written - continue program ***')

 *** Note: tableRowCountGSDR_Temp.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [73]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountGSDR.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:57


In [74]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountGSDR.csv file written - continue program ***')

 *** Note: tableRowCountGSDR.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [75]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ODS_WebApps;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountODS_WebApps.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:58


In [76]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountODS_WebApps.csv file written - continue program ***')

 *** Note: tableRowCountODS_WebApps.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [77]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=PPROD;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountPPROD.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:07:59


In [78]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountPPROD.csv file written - continue program ***')

 *** Note: tableRowCountPPROD.csv file written - continue program ***


#Collect (T) daily and overwrite existing name (Todays data used for loading Greg(DB) tables for automation)

In [79]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Predictive_Analytics;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, s.name as SchemaName, OBJECT_NAME(o.OBJECT_ID) AS TableName, p.row_count as RecordCount
FROM
SYS.objects o JOIN SYS.schemas s
   ON o.schema_id=s.schema_id
        JOIN sys.dm_db_partition_stats p
   ON o.object_id=p.object_id
WHERE o.type LIKE 'U'
ORDER BY TableName

                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableRowCountPredictive_Analytics.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:00


In [80]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableRowCountPredictive_Analytics.csv file written - continue program ***')

 *** Note: tableRowCountPredictive_Analytics.csv file written - continue program ***


(002) Collect data sets that include the query text that can be captured in the past 24 hours

In [81]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=master;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT CONVERT(char(8), GetDate(),112) as TodaysDate
	, DB_NAME(database_id) AS DatabaseName
	, OBJECT_SCHEMA_NAME(ps.object_id, database_id) AS SchemaName
	, OBJECT_NAME(ps.object_id, database_id) AS ObjectName
	, ps.last_execution_time AS LastProcLastExecutionTime
	, qs.last_execution_time AS LastQueryExecutionTime
	, SUBSTRING(st.text, (qs.statement_start_offset/2) + 1,
	 	  ((CASE statement_end_offset 
	 		  WHEN -1 THEN DATALENGTH(ST.text)
	 		  ELSE qs.statement_end_offset END 
	 			  - qs.statement_start_offset)/2) + 1) AS proc_query_text
	, ps.total_logical_reads AS ProcLastTotalReads
	, ps.total_worker_time AS ProcTotalWorkerTime
	, ps.total_elapsed_time AS ProcTotalElapsedTime
	, qs.total_logical_reads AS QueryTotalLogicalReads
	, qs.total_worker_time AS QueryTotalWorkerTime
	, qs.total_elapsed_time AS QueryTotalElapsedTime
FROM sys.dm_exec_procedure_stats AS ps
JOIN sys.dm_exec_query_stats AS qs ON
	qs.sql_handle = ps.sql_handle
CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) as st
--- WHERE
--- 	object_id = OBJECT_ID(N'YourDatabase.dbo.sp_1')
WHERE DB_NAME(database_id) IS NOT NULL --- AND DB_NAME(database_id) in (msdb, master, model, tempdb)
ORDER BY
	 DatabaseName
	,SchemaName
	,ObjectName
	,ps.sql_handle
	,qs.statement_start_offset

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'tableLastRunwSQLtextStatement.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:01


In [82]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: tableLastRunwSQLtextStatement.csv file written - continue program ***')

 *** Note: tableLastRunwSQLtextStatement.csv file written - continue program ***


(003) Collection of details by Chris Lanzi from production and stored daily in the development (Greg) database

In [83]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=GREG;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT [run_date]
      ,[database_name]
      ,[schema_name]
      ,[table_name]
  FROM [Greg].[dbo].[Tables_Unused_Since_Last_Restart]
  Order by run_date desc

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'TablesUsedSinceRunDate-LastUsedTableList.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:10


In [84]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: TablesUsedSinceRunDateLastUsedTableList.csv file written - continue program ***')

 *** Note: TablesUsedSinceRunDateLastUsedTableList.csv file written - continue program ***


(004) Find and collect all table column position precision details

In [85]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ApplicationData;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsApplicationData.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:11


In [86]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsApplicationData.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsApplicationData.csv file written - continue program ***


In [87]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Assessment;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsAssessment.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:13


In [88]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsAssessment.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsAssessment.csv file written - continue program ***


In [89]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Control;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsControl.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:15


In [90]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsControl.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsControl.csv file written - continue program ***


In [91]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsDownstreamFeeds.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:16


In [92]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsDownstreamFeeds.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsDownstreamFeeds.csv file written - continue program ***


In [93]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds_V1.5;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsDownstreamFeeds_V15.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:18


In [94]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsDownstreamFeeds_V15.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsDownstreamFeeds_V15.csv file written - continue program ***


In [95]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GCPS_Operations;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsGCPS_Operations.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:19


In [96]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsGCPS_Operations.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsGCPS_Operations.csv file written - continue program ***


In [97]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsGSDR_Synergy_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:20


In [98]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsGSDR_Synergy_Temp.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsGSDR_Synergy_Temp.csv file written - continue program ***


In [99]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsGSDR_Synergy.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:21


In [100]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsGSDR_Synergy.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsGSDR_Synergy.csv file written - continue program ***


In [101]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsGSDR_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:23


In [102]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsGSDR_Temp.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsGSDR_Temp.csv file written - continue program ***


In [103]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsGSDR.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:25


In [104]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsGSDR.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsGSDR.csv file written - continue program ***


In [105]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ODS_WebApps;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsODS_WebApps.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:26


In [106]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsODS_WebApps.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsODS_WebApps.csv file written - continue program ***


In [107]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=PPROD;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsPPROD.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:27


In [108]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsPPROD.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsPPROD.csv file written - continue program ***


In [109]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Predictive_Analytics;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate, TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, ORDINAL_POSITION, IS_NULLABLE, DATA_TYPE
from INFORMATION_SCHEMA.COLUMNS
Order by TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findTabColPosPrecDetailsPredictive_Analytics.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:29


In [110]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findTabColPosPrecDetailsPredictive_Analytics.csv file written - continue program ***')

 *** Note: findTabColPosPrecDetailsPredictive_Analytics.csv file written - continue program ***


(006) Collect and catalog details about what was last modified over the past 7 days (change to 360 to see a year)

In [111]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ApplicationData;'
                      'Trusted_Connection=yes;')

# Control collection with statement
# WHERE modify_date > GETDATE() - 360

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 7
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360ApplicationData.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:30


In [112]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360ApplicationData.csv file written - continue program ***')

 *** Note: findCrModObjinLast360ApplicationData.csv file written - continue program ***


In [113]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Assessment;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360Assessment.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:31


In [114]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360Assessment.csv file written - continue program ***')

 *** Note: findCrModObjinLast360Assessment.csv file written - continue program ***


In [115]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Control;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360Control.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:32


In [116]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360Control.csv file written - continue program ***')

 *** Note: findCrModObjinLast360Control.csv file written - continue program ***


In [117]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360DownstreamFeeds.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:33


In [118]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360DownstreamFeeds.csv file written - continue program ***')

 *** Note: findCrModObjinLast360DownstreamFeeds.csv file written - continue program ***


In [119]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds_V1.5;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360DownstreamFeeds_V1.5.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:34


In [120]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360DownstreamFeeds_V1.5.csv file written - continue program ***')

 *** Note: findCrModObjinLast360DownstreamFeeds_V1.5.csv file written - continue program ***


In [121]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GCPS_Operations;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 10
ORDER BY modify_date DESC, object_name;  
                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360GCPS_Operations.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:35


In [122]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360GCPS_Operations.csv file written - continue program ***')

 *** Note: findCrModObjinLast360GCPS_Operations.csv file written - continue program ***


In [123]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 10
ORDER BY modify_date DESC, object_name;   

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360GSDR_Synergy_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:36


In [124]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360GSDR_Synergy_Temp.csv file written - continue program ***')

 *** Note: findCrModObjinLast360GSDR_Synergy_Temp.csv file written - continue program ***


In [125]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 10
ORDER BY modify_date DESC, object_name;   

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360GSDR_Synergy.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:37


In [126]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360GSDR_Synergy.csv file written - continue program ***')

 *** Note: findCrModObjinLast360GSDR_Synergy.csv file written - continue program ***


In [127]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360GSDR_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:38


In [128]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360GSDR_Temp.csv file written - continue program ***')

 *** Note: findCrModObjinLast360GSDR_Temp.csv file written - continue program ***


In [129]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;   

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360GSDR.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:40


In [130]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360GSDR.csv file written - continue program ***')

 *** Note: findCrModObjinLast360GSDR.csv file written - continue program ***


In [131]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ODS_WebApps;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360ODS_WebApps.csv'), 'a') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:41


In [132]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=PPROD;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360PPROD.csv'), 'a') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:41


In [133]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360PPROD.csv file written - continue program ***')

 *** Note: findCrModObjinLast360PPROD.csv file written - continue program ***


In [134]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Predictive_Analytics;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate
  ,name AS object_name   
  ,SCHEMA_NAME(schema_id) AS schema_name  
  ,type_desc  
  ,create_date  
  ,modify_date  
FROM sys.objects  
WHERE modify_date > GETDATE() - 8
ORDER BY modify_date DESC, object_name;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findCrModObjinLast360Predictive_Analytics.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:42


In [135]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findCrModObjinLast360Predictive_Analytics.csv file written - continue program ***')

 *** Note: findCrModObjinLast360Predictive_Analytics.csv file written - continue program ***


(007) Capture and catalog all stored procedures (current) in the database(s)

In [136]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ApplicationData;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
 FROM [ApplicationData].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresApplicationData.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:43


In [137]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresApplicationData.csv file written - continue program ***')

 *** Note: ProceduresApplicationData.csv file written - continue program ***


In [138]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Assessment;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [Assessment].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresAssessment.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:44


In [139]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresAssessment.csv file written - continue program ***')

 *** Note: ProceduresAssessment.csv file written - continue program ***


In [140]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Control;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [Control].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresControl.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:45


In [141]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresControl.csv file written - continue program ***')

 *** Note: ProceduresControl.csv file written - continue program ***


In [142]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [DownstreamFeeds].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresDownstreamFeeds.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:46


In [143]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresDownstreamFeeds.csv file written - continue program ***')

 *** Note: ProceduresDownstreamFeeds.csv file written - continue program ***


In [144]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=DownstreamFeeds_V1.5;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [DownstreamFeeds_V1.5].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresDownstreamFeeds_V15.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:47


In [145]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresDownstreamFeeds_V15.csv file written - continue program ***')

 *** Note: ProceduresDownstreamFeeds_V15.csv file written - continue program ***


In [146]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GCPS_Operations;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [GCPS_Operations].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresGCPS_Operations.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:48


In [147]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresGCPS_Operations.csv file written - continue program ***')

 *** Note: ProceduresGCPS_Operations.csv file written - continue program ***


In [148]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [GSDR_Synergy_Temp].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresGSDR_Synergy_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:50


In [149]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresGSDR_Synergy_Temp.csv file written - continue program ***')

 *** Note: ProceduresGSDR_Synergy_Temp.csv file written - continue program ***


In [150]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Synergy;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [GSDR_Synergy].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresGSDR_Synergy.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:51


In [151]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresGSDR_Synergy.csv file written - continue program ***')

 *** Note: ProceduresGSDR_Synergy.csv file written - continue program ***


In [152]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR_Temp;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [GSDR_Temp].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresGSDR_Temp.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:52


In [153]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresGSDR_Temp.csv file written - continue program ***')

 *** Note: ProceduresGSDR_Temp.csv file written - continue program ***


In [154]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [GSDR].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresGSDR.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:53


In [155]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresGSDR.csv file written - continue program ***')

 *** Note: ProceduresGSDR.csv file written - continue program ***


In [156]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=ODS_WebApps;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [ODS_WebApps].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresODS_WebApps.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:55


In [157]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresODS_WebApps.csv file written - continue program ***')

 *** Note: ProceduresODS_WebApps.csv file written - continue program ***


In [158]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=PPROD;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [PPROD].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresPPROD.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:56


In [159]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresPPROD.csv file written - continue program ***')

 *** Note: ProceduresPPROD.csv file written - continue program ***


In [160]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Predictive_Analytics;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT DISTINCT CONVERT(char(8), GetDate(),112) as TodaysDate,
       SPECIFIC_CATALOG, SPECIFIC_SCHEMA, SPECIFIC_NAME, ROUTINE_TYPE, CREATED, LAST_ALTERED
  FROM [Predictive_Analytics].INFORMATION_SCHEMA.ROUTINES
 WHERE ROUTINE_TYPE = 'PROCEDURE' 
   AND LEFT(ROUTINE_NAME, 3) NOT IN ('xp_', 'ms_')
 Order by LAST_ALTERED Desc;  

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'ProceduresPredictive_Analytics.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:57


In [161]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: ProceduresPredictive_Analytics.csv file written - continue program ***')

 *** Note: ProceduresPredictive_Analytics.csv file written - continue program ***


(008) Find and collect all registered DTSX packages in the database (validate against TFS code repository) 

In [162]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Master;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT CONVERT(char(8), GetDate(),112) as TodaysDate
,  J.name AS JobName 
,  S.step_id AS StepNbr
,  S.step_name AS StepName
,  LEFT(REPLACE(REPLACE(S.command, '/FILE "',''),'\"',''), CHARINDEX('.dtsx', REPLACE(REPLACE(S.command, '/FILE "',''),'\"',''))+4) AS SSISPackagePath
FROM  msdb.dbo.sysjobs AS J
INNER JOIN msdb.dbo.sysjobsteps AS S 
ON  J.job_id = S.job_id
WHERE J.enabled = 1 
AND  S.subsystem = 'SSIS'
ORDER BY J.name 
,  S.step_id;

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findtopJobNameStepNum_dtsx.csv'), 'a') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:08:58


In [163]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findtopJobNameStepNum_dtsx.csv file written - continue program ***')

 *** Note: findtopJobNameStepNum_dtsx.csv file written - continue program ***


(009) Find all the current SSMS job failures and review for changes to the system for EDP migration effort

In [164]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=Master;'
                      'Trusted_Connection=yes;')

# The furthest back SQL Server will retain errata on the job runs is day, 120
# change the declare statement back if the results you are seeking are not in the retained file records
# looking at run status = 1 (success) results in nothing tangible for the migration effort 

sql_query = pd.read_sql_query(''' 

declare @date datetime = dateadd(hour, 12, datediff(day, 7, getdate()));

/* PRODODSSQL */
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
select 
    [sJobHistory].[server]
  , [sJOB].[name] as          [JobName]
  , [sJobHistory].step_id
  , [sJobHistory].step_name
  , case
		when [sJobHistory].run_status = 1 then 'Succeeded'
		when [sJobHistory].run_status = 0 then 'Failed'
		else convert(nvarchar(10),[sJobHistory].run_status)
	end	run_status 
  , sJobHistory.[message]
  , [msdb].[dbo].agent_datetime(run_date,run_time) runtime
  , [sJobHistory].run_duration
  , [sJOB].[description] as   [JobDescription]

from [msdb].[dbo].[sysjobs]						as [sJOB]
join [msdb].[dbo].[sysjobhistory]				as [sJobHistory]	on [sjob].job_id					= sJobHistory.job_id
left join [msdb].[sys].[servers]				as [sSVR]			on [sJOB].[originating_server_id]	= [sSVR].[server_id]
left join [msdb].[dbo].[syscategories]			as [sCAT]			on [sJOB].[category_id]				= [sCAT].[category_id]
left join [msdb].[dbo].[sysjobsteps]			as [sJSTP]			on [sJOB].[job_id]					= [sJSTP].[job_id]
																	and [sJOB].[start_step_id]			= [sJSTP].[step_id]
left join [msdb].[sys].[database_principals]	as [sDBP]			on [sJOB].[owner_sid]				= [sDBP].[sid]
left join [msdb].[dbo].[sysjobschedules]		as [sJOBSCH]		on [sJOB].[job_id]					= [sJOBSCH].[job_id]
left join [msdb].[dbo].[sysschedules]			as [sSCH]			on [sJOBSCH].[schedule_id]			= [sSCH].[schedule_id]

where 1=1

and run_status = 0

and [msdb].[dbo].agent_datetime(run_date,run_time) >= @date


/* PRODSISSQL */
union all
select 
    [sJobHistory].[server]
  , [sJOB].[name] as          [JobName]
  , [sJobHistory].step_id
  , [sJobHistory].step_name
  , case
		when [sJobHistory].run_status = 1 then 'Succeeded'
		when [sJobHistory].run_status = 0 then 'Failed'
		else convert(nvarchar(10),[sJobHistory].run_status)
	end	run_status 
  , sJobHistory.[message]
  , [msdb].[dbo].agent_datetime(run_date,run_time) runtime
  , [sJobHistory].run_duration
  , [sJOB].[description] as   [JobDescription]

from prodsissql.[msdb].[dbo].[sysjobs]						as [sJOB]
join prodsissql.[msdb].[dbo].[sysjobhistory]				as [sJobHistory]	on [sjob].job_id					= sJobHistory.job_id
left join prodsissql.[msdb].[sys].[servers]					as [sSVR]			on [sJOB].[originating_server_id]	= [sSVR].[server_id]
left join prodsissql.[msdb].[dbo].[syscategories]			as [sCAT]			on [sJOB].[category_id]				= [sCAT].[category_id]
left join prodsissql.[msdb].[dbo].[sysjobsteps]				as [sJSTP]			on [sJOB].[job_id]					= [sJSTP].[job_id]
																				and [sJOB].[start_step_id]			= [sJSTP].[step_id]
left join prodsissql.[msdb].[sys].[database_principals]		as [sDBP]			on [sJOB].[owner_sid]				= [sDBP].[sid]
left join prodsissql.[msdb].[dbo].[sysjobschedules]			as [sJOBSCH]		on [sJOB].[job_id]					= [sJOBSCH].[job_id]
left join prodsissql.[msdb].[dbo].[sysschedules]			as [sSCH]			on [sJOBSCH].[schedule_id]			= [sSCH].[schedule_id]

where 1=1

and run_status = 0
and [msdb].[dbo].agent_datetime(run_date,run_time) >= @date

order by 
runtime desc

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findSSMSjobFailures.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
    
conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:09:00


In [165]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findSSMSjobFailures.csv file written - continue program ***')

 *** Note: findSSMSjobFailures.csv file written - continue program ***


(010) Collection of table differences compared to partitions found (looking for uniqueness) 

In [166]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRODODSSQL;'
                      'Database=GSDR;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT CONVERT(char(8), GetDate(),112) as TodaysDate,
c.name as SchemaName, t.name as TableName, t.object_id as TableObjectID, s.object_id as PartObjectID, s.row_count as RecordCount
from sys.tables t,
sys.dm_db_partition_stats s,
sys.schemas c
where 
t.schema_id = c.schema_id
AND t.object_id = s.object_id
AND t.type_desc = 'USER_TABLE'
AND t.name not like '%dss%'
AND s.index_id IN (0,1)
ORDER BY 2

                              '''
                              ,conn) 

df = pd.DataFrame(sql_query)
with open(os.path.join(pathT, 'findGSDRtablePartitioningDiffObjectsIDS.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')

conn.close()

now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

Processing Complete:  2021-05-25 07:09:01


In [167]:
#import time
time.sleep(1) # Sleep for 1 seconds
print(' *** Note: findGSDRtablePartitioningDiffObjectsIDS.csv file written - continue program ***')

 *** Note: findGSDRtablePartitioningDiffObjectsIDS.csv file written - continue program ***


In [168]:
now01 = datetime.datetime.now()
print('Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
print(' *** Note: ****.csv files written - this program has completed successfully - review output ***')

Processing Complete:  2021-05-25 07:09:02
 *** Note: ****.csv files written - this program has completed successfully - review output ***
